<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Sources" data-toc-modified-id="Data-Sources-1">Data Sources</a></span></li><li><span><a href="#pyspark.sql-vs-SQL" data-toc-modified-id="pyspark.sql-vs-SQL-2"><code>pyspark.sql</code> vs SQL</a></span><ul class="toc-item"><li><span><a href="#Order-of-execution" data-toc-modified-id="Order-of-execution-2.1">Order of execution</a></span></li><li><span><a href="#Using-SQL-queries-on-a-data-frame" data-toc-modified-id="Using-SQL-queries-on-a-data-frame-2.2">Using SQL queries on a data frame</a></span></li><li><span><a href="#Table-vs-View-concept" data-toc-modified-id="Table-vs-View-concept-2.3">Table vs View concept</a></span></li></ul></li><li><span><a href="#Using-the-Spark-catalog-for-multiple-views" data-toc-modified-id="Using-the-Spark-catalog-for-multiple-views-3">Using the Spark catalog for multiple views</a></span><ul class="toc-item"><li><span><a href="#Data-Source---Backblaze-Data-Set" data-toc-modified-id="Data-Source---Backblaze-Data-Set-3.1">Data Source - Backblaze Data Set</a></span></li></ul></li><li><span><a href="#select-and-where" data-toc-modified-id="select-and-where-4"><code>select</code> and <code>where</code></a></span></li><li><span><a href="#groupby-and-orderby" data-toc-modified-id="groupby-and-orderby-5"><code>groupby</code> and <code>orderby</code></a></span></li><li><span><a href="#Filtering-after-grouping-with-having" data-toc-modified-id="Filtering-after-grouping-with-having-6">Filtering after grouping with <code>having</code></a></span></li><li><span><a href="#Saving-tables/views-using-create" data-toc-modified-id="Saving-tables/views-using-create-7">Saving tables/views using <code>create</code></a></span></li><li><span><a href="#Adding-data-to-table-using-UNION-and-JOIN" data-toc-modified-id="Adding-data-to-table-using-UNION-and-JOIN-8">Adding data to table using <code>UNION</code> and <code>JOIN</code></a></span></li><li><span><a href="#Organizing-code-with-subqueries-and-common-table-expressions-(CTE)" data-toc-modified-id="Organizing-code-with-subqueries-and-common-table-expressions-(CTE)-9">Organizing code with subqueries and common table expressions (CTE)</a></span></li></ul></div>

# Bilingual PySpark: blending Python and SQL

> This chapter is dedicated to using SQL with, and on top of PySpark. I cover how we can move from one language to the other. I also cover how we can use a SQL-like syntax within data frame methods to speed up your code and some of trade-offs you can face. Finally, we blend Python and SQL code together to get the best of both worlds.

## Data Sources

We will be using a periodic table of elements database for the initial section, followed by a public data set provided by BackBlaze, which provides hard drive data and statistics.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
import pyspark.sql.functions as F
import pyspark.sql.types as T
import numpy as np

spark = SparkSession.builder.getOrCreate()

In [2]:
# Read in table of elements data
elements = spark.read.csv(
    "data/Ch07/Periodic_Table_Of_Elements.csv",
    header=True,
    inferSchema=True,
)

# Inspect the data frame
elements.printSchema()

# View the data frame in chunks of 3-4 columns
# column_split = np.array_split(np.array(elements.columns), len(elements.columns) // 3)

# for x in column_split:
#     elements.select(*x).show(3, False)

root
 |-- AtomicNumber: integer (nullable = true)
 |-- Element: string (nullable = true)
 |-- Symbol: string (nullable = true)
 |-- AtomicMass: double (nullable = true)
 |-- NumberofNeutrons: integer (nullable = true)
 |-- NumberofProtons: integer (nullable = true)
 |-- NumberofElectrons: integer (nullable = true)
 |-- Period: integer (nullable = true)
 |-- Group: integer (nullable = true)
 |-- Phase: string (nullable = true)
 |-- Radioactive: string (nullable = true)
 |-- Natural: string (nullable = true)
 |-- Metal: string (nullable = true)
 |-- Nonmetal: string (nullable = true)
 |-- Metalloid: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- AtomicRadius: double (nullable = true)
 |-- Electronegativity: double (nullable = true)
 |-- FirstIonization: double (nullable = true)
 |-- Density: double (nullable = true)
 |-- MeltingPoint: double (nullable = true)
 |-- BoilingPoint: double (nullable = true)
 |-- NumberOfIsotopes: integer (nullable = true)
 |-- Discoverer: s

## `pyspark.sql` vs SQL

### Order of execution

![](./notes/img/order.png)

The code below selects the `phrase` column that contain `"liq"`, then runs groupby and count.

SQL equivalent would be:

```sql
SELECT
  period,
  count(*)
FROM elements
WHERE phase = "liq"
GROUP BY period;
```

In [3]:
elements.where(F.col("phase") == "liq").groupby("period").count().show()

+------+-----+
|period|count|
+------+-----+
|     6|    1|
|     4|    1|
+------+-----+



### Using SQL queries on a data frame

- In order to allow a data frame to be queried via SQL, we need to _register_ them as tables.
- Spark SQL does not have visibility over the variables Python assigns.
- Use `createOrReplaceTempView()` to read a data frame and create a Spark SQL reference.  Functionally equivalent to `CREATE_OR_REPLACE_VIEW` in SQL

In [4]:
# Directly querying a data frame SQL-style does not work
try:
    spark.sql(
        "select period, count(*) from elements where phase='liq' group by period"
    ).show(5)
except AnalysisException as e:
    print(e)

Table or view not found: elements; line 1 pos 29;
'Aggregate ['period], ['period, unresolvedalias(count(1), None)]
+- 'Filter ('phase = liq)
   +- 'UnresolvedRelation [elements]



In [5]:
# Using createOrReplaceTempView

elements.createOrReplaceTempView("elements")

spark.sql(
    "select period, count(*) from elements where phase='liq' group by period"
).show(5)

+------+--------+
|period|count(1)|
+------+--------+
|     6|       1|
|     4|       1|
+------+--------+



### Table vs View concept

> In SQL, they are distinct concepts: the table is materialized in memory and the view is computed on the fly. Spark’s temp views are conceptually closer to a view than a table. Spark SQL also has tables but we will not be using them, preferring reading and materializing our data into a data frame.

## Using the Spark catalog for multiple views

- Spark catalog mainly deals with managing metadata of multiple SQL tables, and their level of caching.
- Catalogs manages views we've registered and drops them.

In [6]:
# Instantiate
spark.catalog

# List tables we've registered
display(spark.catalog.listTables())

# Drop a table
spark.catalog.dropTempView("elements")
display(spark.catalog.listTables())

[Table(name='elements', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

[]

### Data Source - Backblaze Data Set

(Note: Only reading in Q3 data due to local compute)

In [11]:
# Read backblaze data set into a data frame and register a SQL view

DATA_DIRECTORY = "./data/Ch07/"

# q1 = spark.read.csv(
#     DATA_DIRECTORY + "drive_stats_2019_Q1", header=True, inferSchema=True
# )
# q2 = spark.read.csv(
#     DATA_DIRECTORY + "data_Q2_2019", header=True, inferSchema=True
# )
q3 = spark.read.csv(
    DATA_DIRECTORY + "data_Q3_2019", header=True, inferSchema=True
)
# q4 = spark.read.csv(
#     DATA_DIRECTORY + "data_Q4_2019", header=True, inferSchema=True
# )

# Q4 has two more fields than the rest

# q4_fields_extra = set(q4.columns) - set(q1.columns)

# for i in q4_fields_extra:
#     q1 = q1.withColumn(i, F.lit(None).cast(T.StringType()))
#     q2 = q2.withColumn(i, F.lit(None).cast(T.StringType()))
#     q3 = q3.withColumn(i, F.lit(None).cast(T.StringType()))


# Union the data frames

# if you are only using the minimal set of data, use this version
backblaze_2019 = q3

# if you are using the full set of data, use this version
# backblaze_2019 = (
#     q1.select(q4.columns)
#     .union(q2.select(q4.columns))
#     .union(q3.select(q4.columns))
#     .union(q4)
# )

# Setting the layout for each column according to the schema
q = backblaze_2019.select(
    [
        F.col(x).cast(T.LongType()) if x.startswith("smart") else F.col(x)
        for x in backblaze_2019.columns
    ]
)

# Register the view
backblaze_2019.createOrReplaceTempView("backblaze_stats_2019")

In [21]:
backblaze_2019.printSchema()

root
 |-- date: string (nullable = true)
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- capacity_bytes: long (nullable = true)
 |-- failure: integer (nullable = true)
 |-- smart_1_normalized: integer (nullable = true)
 |-- smart_1_raw: integer (nullable = true)
 |-- smart_2_normalized: integer (nullable = true)
 |-- smart_2_raw: integer (nullable = true)
 |-- smart_3_normalized: integer (nullable = true)
 |-- smart_3_raw: integer (nullable = true)
 |-- smart_4_normalized: integer (nullable = true)
 |-- smart_4_raw: integer (nullable = true)
 |-- smart_5_normalized: integer (nullable = true)
 |-- smart_5_raw: integer (nullable = true)
 |-- smart_7_normalized: integer (nullable = true)
 |-- smart_7_raw: long (nullable = true)
 |-- smart_8_normalized: integer (nullable = true)
 |-- smart_8_raw: integer (nullable = true)
 |-- smart_9_normalized: integer (nullable = true)
 |-- smart_9_raw: integer (nullable = true)
 |-- smart_10_normalized: integer (n

## `select` and `where`

Use select and where to show a few hard drives serial numbers that have failed at some point (failure = 1)

In [27]:
# SQL order of operations: 1) select columns, then 2) filter 
spark.sql("select serial_number, model, capacity_bytes from backblaze_stats_2019 where failure = 1").show(5)

# PySpark order of operations: 1) filter, then 2) select columns
backblaze_2019.where("failure=1").select(
    F.col('serial_number'),
    F.col('model'),
    F.col('capacity_bytes')
).show(5)


+-------------+-------------+--------------+
|serial_number|        model|capacity_bytes|
+-------------+-------------+--------------+
|     ZA10MCJ5|  ST8000DM002| 8001563222016|
|     ZCH07T9K|ST12000NM0007|12000138625024|
|     ZCH0CA7Z|ST12000NM0007|12000138625024|
|     Z302F381|  ST4000DM000| 4000787030016|
|     ZCH0B3Z2|ST12000NM0007|12000138625024|
+-------------+-------------+--------------+
only showing top 5 rows

+-------------+-------------+--------------+
|serial_number|        model|capacity_bytes|
+-------------+-------------+--------------+
|     ZA10MCJ5|  ST8000DM002| 8001563222016|
|     ZCH07T9K|ST12000NM0007|12000138625024|
|     ZCH0CA7Z|ST12000NM0007|12000138625024|
|     Z302F381|  ST4000DM000| 4000787030016|
|     ZCH0B3Z2|ST12000NM0007|12000138625024|
+-------------+-------------+--------------+
only showing top 5 rows



## `groupby` and `orderby`

Look at the capacity in gigabytes of the hard drives included in the data, by model.

In [39]:
# Groupby and order in SQL
spark.sql(
    """
    SELECT
        model,
        min(capacity_bytes / pow(1024, 3)) min_GB,
        max(capacity_bytes / pow(1024, 3)) max_GB
    FROM backblaze_stats_2019
    GROUP BY model
    ORDER BY max_GB DESC
"""
).show(5)

+--------------------+--------------------+-------+
|               model|              min_GB| max_GB|
+--------------------+--------------------+-------+
| TOSHIBA MG07ACA14TA|             13039.0|13039.0|
|HGST HUH721212ALE600|             11176.0|11176.0|
|       ST12000NM0117|             11176.0|11176.0|
|       ST12000NM0007|-9.31322574615478...|11176.0|
|HGST HUH721212ALN604|-9.31322574615478...|11176.0|
+--------------------+--------------------+-------+
only showing top 5 rows



In [40]:
# PySpark
backblaze_2019.groupby(F.col("model")).agg(
    F.min(F.col("capacity_bytes") / F.pow(F.lit(1024), 3)).alias("min_GB"),
    F.max(F.col("capacity_bytes") / F.pow(F.lit(1024), 3)).alias("max_GB"),
).orderBy(F.col("max_GB"), ascending=False).show(5)

+--------------------+--------------------+-------+
|               model|              min_GB| max_GB|
+--------------------+--------------------+-------+
| TOSHIBA MG07ACA14TA|             13039.0|13039.0|
|HGST HUH721212ALE600|             11176.0|11176.0|
|       ST12000NM0117|             11176.0|11176.0|
|       ST12000NM0007|-9.31322574615478...|11176.0|
|HGST HUH721212ALN604|-9.31322574615478...|11176.0|
+--------------------+--------------------+-------+
only showing top 5 rows



## Filtering after grouping with `having`

`having` in SQL is a condition block used after grouping is done.

Filter the groupby with only those that have different min_GB and max_GB numbers

In [44]:
spark.sql(
    """
    SELECT
        model,
        min(capacity_bytes / pow(1024, 3)) min_GB,
        max(capacity_bytes / pow(1024, 3)) max_GB
    FROM backblaze_stats_2019
    GROUP BY model
    HAVING min_GB <> max_GB
    ORDER BY max_GB DESC
"""
).show(5)

+--------------------+--------------------+-----------------+
|               model|              min_GB|           max_GB|
+--------------------+--------------------+-----------------+
|       ST12000NM0007|-9.31322574615478...|          11176.0|
|HGST HUH721212ALN604|-9.31322574615478...|          11176.0|
|HGST HUH721010ALE600|-9.31322574615478...|           9314.0|
|       ST10000NM0086|-9.31322574615478...|           9314.0|
|         ST8000DM002|-9.31322574615478...|7452.036460876465|
+--------------------+--------------------+-----------------+
only showing top 5 rows



In [45]:
backblaze_2019.groupby(F.col("model")).agg(
    F.min(F.col("capacity_bytes") / F.pow(F.lit(1024), 3)).alias("min_GB"),
    F.max(F.col("capacity_bytes") / F.pow(F.lit(1024), 3)).alias("max_GB"),
).where(F.col("min_GB") != F.col("max_GB")).orderBy(
    F.col("max_GB"), ascending=False
).show(5)

+--------------------+--------------------+-----------------+
|               model|              min_GB|           max_GB|
+--------------------+--------------------+-----------------+
|       ST12000NM0007|-9.31322574615478...|          11176.0|
|HGST HUH721212ALN604|-9.31322574615478...|          11176.0|
|HGST HUH721010ALE600|-9.31322574615478...|           9314.0|
|       ST10000NM0086|-9.31322574615478...|           9314.0|
|         ST8000DM002|-9.31322574615478...|7452.036460876465|
+--------------------+--------------------+-----------------+
only showing top 5 rows



## Saving tables/views using `create`

- With SQL, prefix query with `CREATE TABLE/VIEW`
    - creating a table will materialize the data
    - creating a view will only keep the query
- With PySpark, just save to variable

Compute the number of days of operation a model has and the number of drive failures it has had

In [56]:
# SQL

spark.catalog.dropTempView('drive_days')
spark.catalog.dropTempView('failures')

spark.sql(
    """
    CREATE TEMP VIEW drive_days AS
        SELECT model, count(*) AS drive_days
        FROM backblaze_stats_2019
        GROUP BY model
""")

spark.sql(
    """
    CREATE TEMP VIEW failures AS
        SELECT model, count(*) AS failures
        FROM backblaze_stats_2019
        WHERE failure = 1
        GROUP BY model
""")

DataFrame[]

In [57]:
# PySpark

drive_days = backblaze_2019.groupBy(F.col("model")).agg(
    F.count(F.col("*")).alias("drive_days")
)

failures = (
    backblaze_2019.where(F.col("failure") == 1)
    .groupBy(F.col("model"))
    .agg(F.count(F.col("*")).alias("failures"))
)

In [59]:
failures.show(5)

+-------------------+--------+
|              model|failures|
+-------------------+--------+
|        ST4000DM000|      72|
|      ST12000NM0007|     365|
|        ST8000DM005|       1|
|TOSHIBA MQ01ABF050M|       5|
|       ST8000NM0055|      50|
+-------------------+--------+
only showing top 5 rows



## Adding data to table using `UNION` and `JOIN`

- SQL `UNION` removes duplicate records, while PySpark doesn't.  
- PySpark `UNION` is equal to SQL `UNION ALL`
- To get SQL `UNION` equivalent with PySpark, run `distinct()` after `union()`

(Note: Not running 2 cells below since I only loaded Q3 data)

In [ ]:
columns_backblaze = ", ".join(q4.columns)

q1.createOrReplaceTempView("Q1")
q1.createOrReplaceTempView("Q2")
q1.createOrReplaceTempView("Q3")
q1.createOrReplaceTempView("Q4")

spark.sql(
    """
    CREATE VIEW backblaze_2019 AS
    SELECT {col} FROM Q1 UNION ALL
    SELECT {col} FROM Q2 UNION ALL
    SELECT {col} FROM Q3 UNION ALL
    SELECT {col} FROM Q4
""".format(
        col=columns_backblaze
    )
)

In [ ]:
backblaze_2019 = (
    q1.select(q4.columns)
    .union(q2.select(q4.columns))
    .union(q3.select(q4.columns))
    .union(q4)
)

Joining `drive_days` and `failures` tables together

In [60]:
spark.sql(
    """
    SELECT
        drive_days.model,
        drive_days,
        failures
    FROM drive_days
    LEFT JOIN failures
    ON
        drive_days.model = failures.model
"""
).show(5)

+-------------+----------+--------+
|        model|drive_days|failures|
+-------------+----------+--------+
|  ST9250315AS|        89|    null|
|  ST4000DM000|   1796728|      72|
|ST12000NM0007|   3212635|     365|
|   ST320LT007|        89|    null|
|  ST8000DM005|      2280|       1|
+-------------+----------+--------+
only showing top 5 rows



In [61]:
drive_days.join(failures, on="model", how="left").show(5)

+-------------+----------+--------+
|        model|drive_days|failures|
+-------------+----------+--------+
|  ST9250315AS|        89|    null|
|  ST4000DM000|   1796728|      72|
|ST12000NM0007|   3212635|     365|
|   ST320LT007|        89|    null|
|  ST8000DM005|      2280|       1|
+-------------+----------+--------+
only showing top 5 rows



## Organizing code with subqueries and common table expressions (CTE)

Take drive_days and failures table definitions and bundle them into a single query using CTE.

In [65]:
spark.sql("""
    WITH drive_days as (
        SELECT
            model,
            count(*) AS drive_days
        FROM backblaze_stats_2019
        GROUP BY model),
    failures as (
        SELECT
            model,
            count(*) AS failures
        FROM backblaze_stats_2019
        WHERE failure = 1
        GROUP BY model)
        
    SELECT
        drive_days.model,
        failures / drive_days failure_rate
    FROM drive_days
    INNER JOIN failures
    ON drive_days.model = failures.model
    ORDER BY failure_rate DESC
""").show(5)

+--------------------+--------------------+
|               model|        failure_rate|
+--------------------+--------------------+
|       ST12000NM0117|0.019305019305019305|
|Seagate BarraCuda...|6.341154090044388E-4|
|  TOSHIBA MQ01ABF050|5.579360828423496E-4|
|         ST8000DM005|4.385964912280702E-4|
|          ST500LM030| 4.19639110365086E-4|
+--------------------+--------------------+
only showing top 5 rows



In [66]:
# CTE sort of similar to python functions
def failure_rate(drive_stats):
    drive_days = drive_stats.groupby(F.col("model")).agg(
        F.count(F.col("*")).alias("drive_days")
    )

    failures = (
        drive_stats.where(F.col("failure") == 1)
        .groupby(F.col("model"))
        .agg(F.count(F.col("*")).alias("failures"))
    )
    answer = (
        drive_days.join(failures, on="model", how="inner")
        .withColumn("failure_rate", F.col("failures") / F.col("drive_days"))
        .orderBy(F.col("failure_rate").desc())
    )
    return answer


failure_rate(backblaze_2019).show(5)

print("drive_days" in dir())

+--------------------+----------+--------+--------------------+
|               model|drive_days|failures|        failure_rate|
+--------------------+----------+--------+--------------------+
|       ST12000NM0117|       259|       5|0.019305019305019305|
|Seagate BarraCuda...|      1577|       1|6.341154090044388E-4|
|  TOSHIBA MQ01ABF050|     44808|      25|5.579360828423496E-4|
|         ST8000DM005|      2280|       1|4.385964912280702E-4|
|          ST500LM030|     21447|       9| 4.19639110365086E-4|
+--------------------+----------+--------+--------------------+
only showing top 5 rows

True
